In [ ]:
from ipynb.fs.full.boundaryCellTraction import *

In [3]:
class boundaryPointTraction(A):

    def __init__(self, boundaries, xy):    # The order matters for boundaries, bottom left corner on the bottom is : ["b", "l"]
        
        sw_S, sw_N, sw_W, sw_E = 0, 0, 0, 0

        if boundaries[0] == "b": sw_S = 1
        if boundaries[0] == "t": sw_N = 1
        if boundaries[0] == "l": sw_W = 1
        if boundaries[0] == "r": sw_E = 1
        
        a_P = (A(xy).a_N*sw_N + A(xy).a_S*sw_S )*2/(Sfy) + (A(xy).a_E*sw_E + A(xy).a_W*sw_W)*2/(Sfx)
        
        a_N = A(xy).a_N*2/(Sfy) # are terms cancel in derivation
        a_S = A(xy).a_S*2/(Sfy)
        a_E = A(xy).a_E*2/(Sfx)
        a_W = A(xy).a_W*2/(Sfx)

        if boundaries[0] == "b":
            self.a_N = a_P
            self.a_P = a_S
        if boundaries[0] == "t":
            self.a_S = a_P
            self.a_P = a_N
        if boundaries[0] == "l":
            self.a_E = a_P
            self.a_P = a_W
        if boundaries[0] == "r":
            self.a_W = a_P
            self.a_P = a_E
    
    def b_diff(boundaries, k, xy, U_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        def N_term():
            
            N_term =( 
                        + A.coef(xy, "N", uv)*(
                            (boundaryPointTraction.corner(boundaries, "E", uv, U_previous, k) - boundaryPointTraction.corner(boundaries, "W", uv, U_previous, k))
                            /dx)
            )

            return N_term

        def S_term():
            
            S_term =(
                        - A.coef(xy, "S", uv)*(
                            (boundaryPointTraction.corner(boundaries, "E", uv, U_previous, k) - boundaryPointTraction.corner(boundaries, "W", uv, U_previous, k))
                            /dx)
                ) 
            return S_term
        def E_term():
            
            E_term =(
                        + A.coef(xy, "E", uv)*(
                            (boundaryPointTraction.corner(boundaries, "N", uv, U_previous, k) - boundaryPointTraction.corner(boundaries, "S", uv, U_previous, k))
                            /dy)
                ) 
            return E_term
        def W_term():
            
            W_term =(
                        - A.coef(xy, "W", uv)*(
                        (boundaryPointTraction.corner(boundaries, "N", uv, U_previous, k) - boundaryPointTraction.corner(boundaries, "S", uv, U_previous, k))
                        /dy)
                )
                
            return W_term

        # This can be tidied up/shortened

        if (boundaries[0] == "b") & (xy == "x") : b_diffusion =  tr_bottom_x  + N_term()
        if (boundaries[0] == "b") & (xy == "y") : b_diffusion =  tr_bottom_y  + N_term()
        if (boundaries[0] == "t") & (xy == "x") : b_diffusion =  tr_top_x + S_term()
        if (boundaries[0] == "t") & (xy == "y") : b_diffusion =  tr_top_y+ S_term()
        if (boundaries[0] == "l") & (xy == "x") : b_diffusion =  tr_left_x + E_term()
        if (boundaries[0] == "l") & (xy == "y") : b_diffusion =  tr_left_y + E_term()
        if (boundaries[0] == "r") & (xy == "x") : b_diffusion =  tr_right_x + W_term()
        if (boundaries[0] == "r") & (xy == "y") : b_diffusion =  tr_right_y + W_term()  

        return b_diffusion
    
    def corner(boundaries, corner_placement, uv, U_previous, k):

        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, U_previous, uv_i)


        if corner_placement == "E":
            corner =  (1/2)*(disp.E + disp.P)
        if corner_placement == "W":
            corner =  (1/2)*(disp.W + disp.P)
        if  corner_placement == "N":
            corner =  (1/2)*(disp.N + disp.P)
        if  corner_placement == "S":
            corner =  (1/2)*(disp.S + disp.P)

        if len(boundaries) > 1:

            if (boundaries[1] == "r") & (corner_placement == "E"):
                    corner =  disp.E
            if (boundaries[1] == "l") & (corner_placement == "W"):
                    corner =  disp.W
            if (boundaries[1] == "t") & (corner_placement == "N"):
                    corner =  disp.N
            if (boundaries[1] == "b") & (corner_placement == "S"):
                    corner =  disp.S
        
        return corner
# boundaryCellTraction.b_diff(["b", "l"], 14, "x", U_previous)
   

In [ ]:
def traction_point_BCs(A_matrix, b_matrix, k, boundaries, xy, U_previous):
    
    #ap
    A_matrix[k,k] = boundaryPointTraction(boundaries, xy).a_P
    #b
    b_matrix[k] =boundaryPointTraction.b_diff(boundaries, k, xy, U_previous)

    if boundaries[0] == "b":
        #an
        A_matrix[k,index(k).n] = - boundaryPointTraction(boundaries, xy).a_N
    if boundaries[0] == "t":
        #as
        A_matrix[k,index(k).s] = - boundaryPointTraction(boundaries, xy).a_S
    if boundaries[0] == "l":
        #ae
        A_matrix[k, index(k).e] = - boundaryPointTraction(boundaries, xy).a_E
    if boundaries[0] == "r":
        #aw
        A_matrix[k, index(k).w] = -  boundaryPointTraction(boundaries, xy).a_W

    return A_matrix

def traction_point_BCs_b(b_matrix, k, edges, xy, U_previous, time):
    
    #b
    b_matrix[k] =boundaryPointTraction.b_diff(edges, k, xy, U_previous, time)

    return b_matrix
